### 기본 RNN 구현

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, SimpleRNN

In [4]:
model = Sequential(name = "simple rnn")

model.add(Input(shape = (10,5)))
model.add(SimpleRNN(20))
model.add(Dense(10, activation = 'softmax'))

model.summary()

Model: "simple rnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 20)             │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 730 (2.85 KB)

 Trainable params: 730 (2.85 KB)

 Non-trainable params: 0 (0.00 B)

### RNN기반 시계열 데이터 예측

In [15]:
import warnings
import os
import numpy
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import RNN, Dense, Input, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

### 임의 데이터 생성

In [17]:
def create_classification_data(n_samples=1000, n_features=2, n_informative=2,
                               n_redundant=0, n_clusters_per_class=1, n_classes=2, random_state=42):
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_informative,
        n_redundant=n_redundant,
        n_clusters_per_class=n_clusters_per_class,
        n_classes=n_classes,
        random_state=random_state
    )
    # DataFrame으로 변환하여 보기 쉽게 만듭니다.
    # 컬럼 이름은 'feature_0', 'feature_1', ..., 'target'
    feature_names = [f'feature_{i}' for i in range(n_features)]
    df = pd.DataFrame(X, columns=feature_names)
    df['target'] = y
    return df, X, y

print("--- 분류 데이터 생성 시작 ---")
# 2개의 특징을 가진 이진 분류 데이터 생성 (2개의 클래스)
# 각 클래스 내에 1개의 클러스터가 있어 비교적 선형적으로 분류 가능합니다.
df_classification, X_classification, y_classification = create_classification_data(
    n_samples=1000,       # 1000개의 데이터 포인트
    n_features=2,         # 2개의 특징 (x, y 좌표처럼 시각화하기 좋음)
    n_informative=2,      # 2개의 특징 모두 분류에 유용함
    n_redundant=0,        # 중복 특징 없음
    n_clusters_per_class=1, # 각 클래스는 하나의 명확한 군집을 형성
    n_classes=2,          # 2개의 클래스 (0 또는 1)
    random_state=42
)

print(f"생성된 DataFrame 형태: {df_classification.shape}")
print(f"X (특징) 형태: {X_classification.shape}")
print(f"y (타겟 클래스) 형태: {y_classification.shape}")
print("\n--- 생성된 데이터의 처음 5개 행: ---")
print(df_classification.head())

--- 분류 데이터 생성 시작 ---
생성된 DataFrame 형태: (1000, 3)
X (특징) 형태: (1000, 2)
y (타겟 클래스) 형태: (1000,)

--- 생성된 데이터의 처음 5개 행: ---
   feature_0  feature_1  target
0   0.601034   1.535353       1
1   0.755945  -1.172352       0
2   1.354479  -0.948528       0
3   3.103090   0.233485       0
4   0.753178   0.787514       1


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_classification, y_classification, test_size = 0.2, random_state = 42)

scaler = StandardScaler()

X_train_scaled_cls = scaler.fit_transform(X_train)
X_test_scaled_cls = scaler.transform(X_test)

print("\n--- 스케일링 결과 (처음 5개 행) ---")
print("X_train_scaled_cls:\n", X_train_scaled_cls[:5])
print("X_test_scaled_cls:\n", X_test_scaled_cls[:5])


--- 스케일링 결과 (처음 5개 행) ---
X_train_scaled_cls:
 [[ 0.42103266  0.43750578]
 [-0.30480465 -0.86556712]
 [ 0.68552745  0.16742658]
 [-2.21033447 -1.8230404 ]
 [ 0.58482223 -0.47622049]]
X_test_scaled_cls:
 [[ 0.16785555  0.43832313]
 [-1.6602748  -1.59042596]
 [-0.04814766 -0.43506505]
 [-0.65959026 -1.33810021]
 [-1.99622754 -1.55888685]]


In [21]:
X_train_final = X_train_scaled_cls.reshape(X_train_scaled_cls.shape[0], X_train_scaled_cls.shape[1], 1)
X_test_final = X_test_scaled_cls.reshape(X_test_scaled_cls.shape[0], X_test_scaled_cls.shape[1], 1)

In [22]:
X_train_final.shape, X_test_final.shape # 데이터 개수, 시퀀스 길이, 특징 수

((800, 2, 1), (200, 2, 1))

In [23]:
sequence_length = 50
num_sequences = 1000

model = Sequential()

model.add(Input(shape = (sequence_length, 1)))
# 현재 SimpleRNN 층을 다음 RNN 층에 전달
model.add(SimpleRNN(units = 64, activation = 'relu', return_sequences = True))
model.add(SimpleRNN(units = 128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [24]:
model.compile(
  optimizer=tf.optimizers.Adam(learning_rate=1e-4),
  loss='binary_crossentropy',
  metrics=['accuracy'])

In [30]:
callbacks = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

# 모델 훈련
print("\n--- 모델 훈련 시작 ---")
history = model.fit(
    X_train_final, y_train,
    epochs=50,             # 최대 에포크 수
    batch_size=32,          # 배치 크기
    validation_split=0.1,   # 훈련 데이터의 10%를 검증 데이터로 사용
    callbacks=callbacks, # EarlyStopping 콜백 적용
    verbose=1               # 훈련 진행 상황 표시
)
print("--- 모델 훈련 완료 ---")


--- 모델 훈련 시작 ---
Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9234 - loss: 0.1652 - val_accuracy: 0.9500 - val_loss: 0.1496
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9499 - loss: 0.1258 - val_accuracy: 0.9375 - val_loss: 0.1477
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9454 - loss: 0.1326 - val_accuracy: 0.9375 - val_loss: 0.1505
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9446 - loss: 0.1360 - val_accuracy: 0.9375 - val_loss: 0.1511
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9313 - loss: 0.1755 - val_accuracy: 0.9375 - val_loss: 0.1504
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9288 - loss: 0.1603 - val_accuracy: 0.9375 - val_loss: 0.1484
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9370 - loss: 0.1404 - val_accuracy: 0.9250 - val_loss: 0.1487
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9382 - loss: 0.1425 - val_accuracy: 

In [31]:
# 모델 평가
loss,acc = model.evaluate(X_test_final, y_test, verbose=0)
print(f"loss: {loss:.3f} | acc: {acc:.3f}")

loss: 0.198 | acc: 0.915


### LSTM 모델을 활용한 자연어 감정 분류 

In [36]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, TextVectorization, Embedding, LSTM
from tensorflow.keras.optimizers import Adam

### Tensorflow Dataset 구성 (10점)
- `keras.utils.text_dataset_from_directory()`를 활용하여 저장된 텍스트 파일로부터 tensorflow Dataset 생성

In [ ]:
train_data_path = "/mnt/elice/dataset/train"
test_data_path = "/mnt/elice/dataset/test"

batch_size = 64

raw_train_dataset = tf.keras.preprocessing.text_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    class_names=['neg', 'pos'],
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)

raw_val_dataset = tf.keras.preprocessing.text_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='binary',
    batch_size=batch_size,
    class_names=['neg', 'pos'],
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='validation'
)

raw_test_dataset = tf.keras.preprocessing.text_dataset_from_directory(
    test_data_path,
    labels=None,
    label_mode=None,
    batch_size=batch_size,
    shuffle=False,
)

"\nraw_train_dataset = tf.keras.preprocessing.text_dataset_from_directory(\n    train_data_path,\n    labels='inferred',\n    label_mode='binary',\n    batch_size=batch_size,\n    class_names=['neg', 'pos'],\n    shuffle=True,\n    seed=42,\n    validation_split=0.2,\n    subset='training'\n)\n\nraw_val_dataset = tf.keras.preprocessing.text_dataset_from_directory(\n    train_data_path,\n    labels='inferred',\n    label_mode='binary',\n    batch_size=batch_size,\n    class_names=['neg', 'pos'],\n    shuffle=True,\n    seed=42,\n    validation_split=0.2,\n    subset='validation'\n)\n\nraw_test_dataset = tf.keras.preprocessing.text_dataset_from_directory(\n    test_data_path,\n    labels=None,\n    label_mode=None,\n    batch_size=batch_size,\n    shuffle=False,\n)\n"

In [ ]:
# 현재 imdb 데이터셋을 keras에서 지원해주므로 로드
# from tensorflow.keras.datasets import imdb

# max_words = 500 # 사전에 사용할 최대 단어 수, 상위 500개만 사용함 

# (train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=max_words)
# print(train_input.shape, test_input.shape)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(25000,) (25000,)


### 데이터 전처리 (10점)
- 텍스트 소문자 변환
- 불용어(stopword)제거
  - 여기서는 문장부호 및 **br** 태그만 삭제

In [34]:
import re
import string

def text_standrization(input_text):
  input_text_lowercase = tf.strings.lower(input_text)
  input_text_processed = tf.strings.regex_replace(input_text_lowercase, "<br />"," ")
  punctuation_regex = f"[{re.escape(string.punctuation)}]" # 불용어 정의
  
  input_text_processed = tf.strings.regex_replace(input_text_processed, punctuation_regex, "")
  return input_text_processed

### 텍스트 데이터 인코딩 (20점)

전처리 완료한 데이터를 정해진 규칙에 따라 숫자의 배열 (벡터)로 변환합니다.

Keras의 `TextVectorization` 레이어를 활용해 가장 자주 등장하는 단어 순으로 숫자를 부여하고, 벡터로 변환하세요.

- TextVectorization 레이어의 `standardize` 인자에는 전처리 함수인 text_standrization을 사용합니다.
- max_tokens 인자에는 `max_words` 변수의 값을 사용합니다.
- output_sequence_length 인자에는 `max_len` 변수의 값을 사용합니다.

In [ ]:
# max_words: 훈련 데이터에서 가장 자주 등장하는 상위 max_words 개의 단어만 어휘 사전에 포함시키고, 나머지는 Unknown로 처리하는 단어의 개수
# embedding_dim: 각 단어를 표현하는 벡터의 차원
# max_len: 각 입력 시퀀스의 최대 단어 수

max_words = 500
embedding_dim = 150
max_len = 256

text_vectorization_layer = TextVectorization(
  standardize = text_standrization,
  max_tokens = max_words,
  output_sequence_length = max_len,
)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return text_vectorization_layer(text), label

def vectorize_text_test(text):
    text = tf.expand_dims(text, -1)
    return text_vectorization_layer(text)
  
# 각 데이터셋에 vectorize_text 매핑
train_dataset = raw_train_dataset.map(vectorize_text)
val_dataset = raw_val_dataset.map(vectorize_text)
test_dataset = raw_test_dataset.map(vectorize_text_test)

### 모델 정의 및 컴파일 (20점)

- A. Embedding Layer
  - 벡터화된 자연어 데이터를 128차원의 벡터로 변환하는 Embedding Layer를 사용합니다.

- B. LSTM Classifier
  - 단일 LSTM Layer를 사용하며, 은닉층의 차원은 32입니다.
  - 출력층의 차원은 1입니다.

In [39]:
model = Sequential()
model.add(Input(shape=(None,)))
# [지시사항 4] LSTM 모델을 구성하세요.
# Hint. model.add()를 3번 사용합니다.
model.add(Embedding(max_words, 128))
model.add(LSTM(32))
model.add(Dense(1, activation = "sigmoid"))

### 모델 학습 (20점)

- epochs는 30으로 설정합니다.
- 학습 데이터로는 train_dataset을 사용합니다.
- 검증 데이터로는 val_dataset을 사용합니다.
- 미리 정의된 callbacks 배열을 콜백 함수로 등록합니다.

In [ ]:
model.compile(
  optimizer=tf.optimizers.Adam(learning_rate=1e-3),
  loss='binary_crossentropy',
  metrics=['accuracy'])

model.summary()

### 모델 평가 (20점)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

hist = model.fit(
    X_train, y_train, epochs = 30, batch_size = 32, callbacks = callbacks, validation_data = (val_train, val_test))